In [ ]:
import monai
import torch
import medmnist
from acsconv.converters import ACSConverter
from medcam import medcam
from medmnist import INFO, Evaluator

from experiments.MedMNIST3D.models import ResNet18
from experiments.MedMNIST3D.utils import Transform3D, model_to_syncbn

In [ ]:
MODEL_PATH = "./output/organmnist3d/resnet18/best_model.pth"
BATCH_SIZE = 1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset_name: str = "organmnist3d"
download: bool = True

info = INFO[dataset_name]
num_classes = len(info["label"])
DataClass = getattr(medmnist, info['python_class'])

train_transform = Transform3D()
eval_transform = Transform3D()

test_dataset = DataClass(split='test', transform=eval_transform, download=download, as_rgb=False)
test_loader = monai.data.DataLoader(dataset=test_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=False)
model = ResNet18(num_classes=num_classes, in_channels=1)
model = model_to_syncbn(ACSConverter(model))
model = medcam.inject(model, output_dir="attention_maps", save_maps=True)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device)['net'], strict=True)

In [ ]:
image_batch, batch_labels = next(test_loader.__iter__())
model.eval()
tep = model(image_batch)
print(tep)